In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch
import os
from tqdm.notebook import tqdm
from torch.cuda.amp import autocast, GradScaler


### 1) Load Dataset and Split

In [ ]:
from datasets import load_dataset

ds = load_dataset("garythung/trashnet", cache_dir="/content/cache")


In [ ]:
print(ds)

In [ ]:
# HF Dataset üzerinde Stratify benzeri split (class distribution korunur)
ds_train_test = ds['train'].train_test_split(test_size=0.2, seed=42)
ds_train_valid = ds_train_test['train'].train_test_split(test_size=0.125, seed=42)  # ~70/15/15

train_raw = ds_train_valid['train']
val_raw   = ds_train_valid['test']
test_raw  = ds_train_test['test']


Validation veri kümesi hiperparametre ayarı, erken durdurma ve model seçimi için kullanılır.

### 2) Transforms

In [ ]:
INPUT_SIZE = (64, 64)

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),   # gri tonlamaya dönüştürme
    transforms.Resize(INPUT_SIZE, antialias=True),
    transforms.ToTensor(),                         # pikselleri [0, 1] aralığına getir
    transforms.Normalize(mean=[0.5], std=[0.5])    # [-1, 1] aralığına normalleştirme
])

In [ ]:
val_test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(INPUT_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


Transform ile Data Augmentation yapılarak modelin daha çok veri ile ile beslenip daha iyi genelleme yapması amaçlanıyor.

### 3) Dataset Wrapper

In [ ]:
# HF Dataset'i PyTorch DataLoader ile kullanılabilir hale getirme
# Bu, DataLoader'ın PyTorch modeline uygun şekilde veri sağlamasını sağlar.
class HFDatasetWrapper(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.ds = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        image = item['image']   # PIL image
        label = item['label']
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Pytorch Dataset'leri
train_ds = HFDatasetWrapper(train_raw, transform=train_transform)
val_ds   = HFDatasetWrapper(val_raw,   transform=val_test_transform)
test_ds  = HFDatasetWrapper(test_raw,  transform=val_test_transform)


### 4) DataLoaders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/veri_kaynak /content/


In [ ]:
torch.cuda.empty_cache()


batch_size = 24


num_workers = 2

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                         num_workers=num_workers, pin_memory=True,
                         persistent_workers=True, drop_last=True, prefetch_factor=2)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True,
                         persistent_workers=False)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True,
                         persistent_workers=False)



### 5) Model

In [ ]:
class SimpleTrashCNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=6, dropout=0.3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 128 -> 64

            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 64 -> 32
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),   # 32x32 -> 1x1
            nn.Flatten(),                  # 32 * 1 * 1 = 32
            nn.Dropout(p=dropout),
            nn.Linear(32, num_classes)     # <-- Burada 64 yerine 32
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


### 6) Loss, Optimizer, Scheduler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)
model = SimpleTrashCNN(in_channels=1, num_classes=6).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)


### 7) Early Stopping Class

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, save_path="best_model.pth"):
        self.patience = patience
        self.counter = 0
        self.best_loss = float("inf")
        self.early_stop = False
        self.save_path = save_path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), self.save_path)
            print(f"Model saved: {self.save_path}")
        else:
            self.counter += 1
            print(f"Early stopped counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
early_stopping = EarlyStopping(patience=5, save_path="/content/drive/MyDrive/best_model.pth")

### 8) Train, Eval, Test Functions

In [ ]:
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # Daha verimli CPU aktarımı
            all_preds.append(preds.detach().cpu())
            all_labels.append(labels.detach().cpu())

    avg_loss = running_loss / total
    acc = correct / total

    # Liste yerine tek tensor halinde döndürmek daha verimli
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    return avg_loss, acc, all_preds, all_labels


In [ ]:
from torch.amp import autocast
scaler = torch.amp.GradScaler('cuda')

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, early_stopping, epochs=10):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    scaler = GradScaler()

    for epoch in tqdm(range(epochs), desc="Epochs", dynamic_ncols=True):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        batch_bar = tqdm(train_loader, leave=False, dynamic_ncols=True, desc=f"Epoch {epoch+1}")

        for images, labels in batch_bar:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad()

            with autocast(device_type=device.type, dtype=torch.float16):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            batch_bar.set_postfix({'Loss': f'{loss.item():.4f}'})

        train_loss = running_loss / total
        train_acc = correct / total

        # Her 2 epoch'ta bir validation
        if epoch % 2 == 0:
            val_loss, val_acc, _, _ = evaluate_model(model, val_loader, criterion, device)
            scheduler.step(val_loss)
        else:
            val_loss, val_acc = None, None

        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} "
              f"{f'Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f}' if val_loss is not None else '(Validation skipped)'}")

        if val_loss is not None:
            early_stopping(val_loss, model)
            if early_stopping.early_stop:
                print("Early stopping triggered. Stopping training.")
                break

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    return train_losses, train_accs, val_losses, val_accs

In [ ]:
def test_model(model, test_loader, device, model_path="/content/drive/MyDrive/best_model.pth"):
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f"Best model loaded : {model_path}")
    else:
        print("Model not found, using current model state.")

    test_loss, test_acc, test_preds, test_labels = evaluate_model(model, test_loader, nn.CrossEntropyLoss(), device)
    print(f"Test Accuracy: {test_acc:.4f}")
    return test_preds, test_labels

### 9) Training and Test

In [ ]:
train_losses, train_accs, val_losses, val_accs = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler,
    device, early_stopping, epochs=10
)


In [ ]:
test_preds, test_labels = test_model(model, test_loader, device)

### 10) Visualization

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


In [ ]:
def report_results(preds, labels, class_names=None):
    # Tensorları numpy array'e çevir
    preds = preds.cpu().numpy() if hasattr(preds, 'cpu') else preds
    labels = labels.cpu().numpy() if hasattr(labels, 'cpu') else labels

    # Confusion Matrix
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()

    # Classification Report
    print("\nClassification Report:\n")
    print(classification_report(labels, preds, target_names=class_names))

In [ ]:
class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

report_results(test_preds, test_labels, class_names)


In [ ]:
# Veri yükleme hızını test et
import time
start = time.time()
for i, (images, labels) in enumerate(train_loader):
    if i == 5:  # İlk 5 batch
        break
    print(f"Batch {i+1} loaded in {time.time() - start:.2f}s")
    start = time.time()